In [ ]:
import pandas as pd
import polars as pl

# from cluster import embed_items, create_cluster
import requests
from bs4 import BeautifulSoup


def parse_historical_table(html_content, session=None):
    """
    Parses the messy HTML table into a structured list of dictionaries.
    """
    if session is None:
        session = requests
    next_root = "https://www.british-history.ac.uk"
    html = session.get(next_root + html_content, timeout=10).text
    soup = BeautifulSoup(html, "html.parser")
    next_row = soup.find("div", {"id": "block-componentpager"})
    rows = soup.find(class_="table-wrap").find("table").find_all(["th", "td"])
    idx = [
        i
        for i, element in enumerate(rows)
        if element.name == "th" and element.get_text(strip=True)
    ]
    citation = soup.find(class_="chicago").text.strip()
    cols = ["date", "text", "citation"]
    lines = []
    for index, i in enumerate(idx):
        row = []
        row.append(rows[i].text)
        if index == idx.index(idx[-1]):
            row.append("\n".join(t.text for t in rows[i + 1 :]))
        else:
            row.append("\n".join(t.text for t in rows[i + 1 : idx[index + 1]]))
        row.append(citation)
        lines.append(row)

    df1 = pl.DataFrame(data=lines, schema=cols)
    df1 = df1.with_columns(pl.col("text").str.replace_all(r"\n", " "))
    # df1["text"] = df1["text"].str.replace("\n", "")

    return df1

In [ ]:
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry


def get_session():
    """Session with retries and backoff."""
    session = requests.Session()
    retry = Retry(total=3, backoff_factor=1, status_forcelist=[429, 500, 502, 503, 504])
    adapter = HTTPAdapter(max_retries=retry)
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    return session


session = get_session()
# from cluster import embed_items
# url_num = 4

with open("data_links.txt", "r") as f:
    data_links = f.read().splitlines()

# data_links = data_links[:url_num]

dfs = []
for url in data_links:
    df = parse_historical_table(url, session=session)
    dfs.append(df)
    print(f"finished {url}")
table = pl.concat(dfs, how="vertical")
display(table.shape)

finished /cal-state-papers/venice/vol1/pp1-3
finished /cal-state-papers/venice/vol1/pp3-39
finished /cal-state-papers/venice/vol1/pp39-52
finished /cal-state-papers/venice/vol1/pp52-61
finished /cal-state-papers/venice/vol1/pp61-63
finished /cal-state-papers/venice/vol1/pp63-65
finished /cal-state-papers/venice/vol1/pp65-74
finished /cal-state-papers/venice/vol1/pp74-92
finished /cal-state-papers/venice/vol1/pp92-126
finished /cal-state-papers/venice/vol1/pp126-140
finished /cal-state-papers/venice/vol1/pp141-159
finished /cal-state-papers/venice/vol1/pp159-203
finished /cal-state-papers/venice/vol1/pp203-226
finished /cal-state-papers/venice/vol1/pp226-252
finished /cal-state-papers/venice/vol1/pp252-266
finished /cal-state-papers/venice/vol1/pp267-276
finished /cal-state-papers/venice/vol1/pp276-286
finished /cal-state-papers/venice/vol1/pp286-289
finished /cal-state-papers/venice/vol1/pp289-291
finished /cal-state-papers/venice/vol1/pp291-295
finished /cal-state-papers/venice/vol1/p

(53939, 3)

In [ ]:
# Run this in your notebook
print(f"Number of texts: {len(table['text'])}")
print(
    f"Average length: {table.select(pl.col('text')).str.split(' ').str.len().mean():.0f} words"
)
print(
    f"Total characters: {table.select(pl.col(['text'])).str.split(' ').str.len().sum():,}"
)

Number of texts: 53939
Average length: 283 words
Total characters: 15,273,702


In [ ]:
# table.to_csv("./data/state_papers_ven-for.csv", index=False)

In [2]:
t = pl.read_csv("./data/state_papers_ven-for.csv")
t.head()

date,text,citation
str,str,str
"""1202. Oct.""","""1. Baldwin Count of Flanders a…","""'Venice: 1202-1295', in Calend…"
"""1224. 13 Sept.""","""2. Doge Pietro Ziani and his s…","""'Venice: 1202-1295', in Calend…"
"""1265. Nov. 6. “Fractus.” “Deli…","""3. Tariff of Duties on Cloths,…","""'Venice: 1202-1295', in Calend…"
"""1272. Feb. 15. “Comune I.” “De…","""4. “Cambium.”Concerning mercha…","""'Venice: 1202-1295', in Calend…"
"""1273. Dec. 13. “ Comune I.” “D…","""5. “Cambium.”Concerning mercha…","""'Venice: 1202-1295', in Calend…"


In [3]:
t.shape

(53939, 3)

In [4]:
t = t.with_columns(
    pl.col("date")
    .str.slice(0, 10)
    .str.extract(r"(\d{4})\b")
    .cast(pl.Int64)
    .alias("year")
    .forward_fill()
)
t.tail(20)

date,text,citation,year
str,str,str,i64
"""July 27.""","""Sir Francis Vere to Walsingham…","""'Elizabeth: July 1589, 21-31',…",1588
"""July 27.""","""Sir Francis Vere to Willoughby…","""'Elizabeth: July 1589, 21-31',…",1588
"""July 27.""","""Captain E. Bannaster to Burghl…","""'Elizabeth: July 1589, 21-31',…",1588
"""July 27.""","""Edward Barton to [Walsingham?]…","""'Elizabeth: July 1589, 21-31',…",1588
"""[July 28.]""","""[A. de Chatte, governor of Die…","""'Elizabeth: July 1589, 21-31',…",1588
…,…,…,…
"""July 31.""","""Richard Tomson to Walsingham.F…","""'Elizabeth: July 1589, 21-31',…",1588
"""[July 31.]/[Aug. 10.]]""","""Ottywell Smyth to [Walsingham?…","""'Elizabeth: July 1589, 21-31',…",1588
"""[July.]""","""Sir Roger Williams to the Lord…","""'Elizabeth: July 1589, 21-31',…",1588


In [5]:
t.select(pl.col("year")).describe()

statistic,year
str,f64
"""count""",53939.0
"""null_count""",0.0
"""mean""",1587.161887
"""std""",40.198828
"""min""",1202.0
"""25%""",1563.0
"""50%""",1583.0
"""75%""",1617.0
"""max""",1894.0


In [6]:
month_map = {
    "jan": 1,
    "feb": 2,
    "mar": 3,
    "apr": 4,
    "may": 5,
    "jun": 6,
    "jul": 7,
    "aug": 8,
    "sep": 9,
    "oct": 10,
    "nov": 11,
    "dec": 12,
}

t = t.with_columns(
    [
        pl.col("date")
        .str.to_lowercase()
        .str.extract(r"(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)")
        .replace(month_map)
        .cast(pl.Int64)
        .fill_null(1)
        .alias("month"),
        pl.col("date")
        .str.extract(r"(?:^|\D)(\d{4})(?:\D|$)")
        .cast(pl.Int64)
        .forward_fill()
        .alias("year"),
    ]
)
t.select(pl.col("year").is_null()).sum()

year
u32
0


In [7]:
t = t.with_columns(
    pl.col("citation")
    .str.slice(0, 50)
    .str.extract_all(r"(\d{4})")
    .alias("citation_years")
)

t = t.with_columns(
    pl.when(pl.col("citation_years").list.len() == 1)
    .then(
        pl.when(pl.col("year") != pl.col("citation_years").list.first().cast(pl.Int64))
        .then(pl.col("citation_years").list.first().cast(pl.Int64))
        .otherwise(pl.col("year"))
    )
    .otherwise(pl.col("year"))
    .alias("year_adjusted")
)

In [8]:
t.filter(pl.col("citation_years").list.len() > 1).sample(n=15)

date,text,citation,year,month,citation_years,year_adjusted
str,str,str,i64,i64,list[str],i64
"""Dec 10. Sforza Archives, Milan…","""359. King Henry VI. to Pope Pi…","""'Venice: 1451-1460', in Calend…",1460,12,"[""1451"", ""1460""]",1460
"""Dec. 9. Senato Mar. v. iv. p. …","""303. Decree of the Senate.By a…","""'Venice: 1451-1460', in Calend…",1451,12,"[""1451"", ""1460""]",1451
"""Aug. 31. In the Library of the…","""458. Protest of a Bill of Exch…","""'Venice: 1471-1480', in Calend…",1476,8,"[""1471"", ""1480""]",1476
"""1455. Jan. 2. In the Library o…","""321. Protest of a Bill of Exch…","""'Venice: 1451-1460', in Calend…",1455,1,"[""1451"", ""1460""]",1455
"""Oct. 27. St. Mark's Library.""","""536. John Balue, Cardinal of A…","""'Venice: 1486-1490', in Calend…",1488,10,"[""1486"", ""1490""]",1488
…,…,…,…,…,…,…
"""June 8. Deliberazioni Senato S…","""467. The Doge and Senate to Ed…","""'Venice: 1471-1480', in Calend…",1479,6,"[""1471"", ""1480""]",1479
"""Oct. 28. “Secreta Senato Delib…","""243. The Senate to “Ser” Frede…","""'Venice: 1421-1430', in Calend…",1430,10,"[""1421"", ""1430""]",1430
"""March 11. Senato Mar. v. 13. p…","""608. Decree of the Senate.Desi…","""'Venice: 1491-1495', in Calend…",1491,3,"[""1491"", ""1495""]",1491


In [9]:
t = t.with_columns(
    pl.col("date")
    .str.extract(r"(?:\b|^)(\d{1,2})(?:\b|$)")
    .cast(pl.Int64)
    .fill_null(1)
    .alias("day"),
    pl.col("date")
    .str.extract(r"(?:\b|^)(\d{1,2})(?:\b|$)")
    .is_not_null()
    .alias("has_day"),
    pl.col("date")
    .str.to_lowercase()
    .str.extract(r"(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)")
    .is_not_null()
    .alias("has_month"),
)

In [10]:
t = t.with_columns(
    pl.when(pl.col("has_day"))
    .then(pl.lit("day"))
    .otherwise(
        pl.when(pl.col("has_month")).then(pl.lit("month")).otherwise(pl.lit("year"))
    )
    .alias("precision"),
)

In [11]:
t = t.drop(["year", "has_day", "has_month"])
t = t.rename({"year_adjusted": "year"})
t.sample(n=10)

date,text,citation,month,citation_years,year,day,precision
str,str,str,i64,list[str],i64,i64,str
"""Oct. 15.""","""855. Another copy of the above…","""'Elizabeth: October 1562, 11-1…",10,"[""1562""]",1562,15,"""day"""
"""July 29.""","""2378. Arrest of English Ships …","""'Elizabeth: July 1568', in Cal…",7,"[""1568""]",1568,29,"""day"""
"""March 31. Sanuto Diaries, v. 1…","""437. Lodovico Falier to the Si…","""'Venice: March 1529', in Calen…",3,"[""1529""]",1529,31,"""day"""
"""Dec. 20. Original Despatch, Ve…","""455. Piero Priuli, Venetian Am…","""'Venice: December 1605', in Ca…",12,"[""1605""]",1605,20,"""day"""
""" July. ""","""Draft for the following paper,…","""'Elizabeth: July 1586, 1-10', …",7,"[""1586""]",1586,1,"""month"""
""" Feb. 8. ""","""539. DUKE of ANJOU to PALLAVIC…","""'Elizabeth: February 1582, 1-1…",2,"[""1582""]",1582,8,"""day"""
"""March 12.Senato,Secreta.Dispac…","""766. GIROLAMO LANDO, Venetian …","""'Venice: March 1621, 12-20', i…",3,"[""1621""]",1621,12,"""day"""
"""Oct. 5.""","""605. The Queen to Lee.Means to…","""'Elizabeth: October 1560, 1-5'…",10,"[""1560""]",1560,5,"""day"""
"""Jan. 22.""","""75. Guido Giannetti to the Que…","""'Elizabeth: January 1564, 16-3…",1,"[""1564""]",1564,22,"""day"""


In [12]:
t = t.with_columns(pl.row_index())
t.head()

date,text,citation,month,citation_years,year,day,precision,index
str,str,str,i64,list[str],i64,i64,str,u32
"""1202. Oct.""","""1. Baldwin Count of Flanders a…","""'Venice: 1202-1295', in Calend…",10,"[""1202"", ""1295""]",1202,1,"""month""",0
"""1224. 13 Sept.""","""2. Doge Pietro Ziani and his s…","""'Venice: 1202-1295', in Calend…",9,"[""1202"", ""1295""]",1224,13,"""day""",1
"""1265. Nov. 6. “Fractus.” “Deli…","""3. Tariff of Duties on Cloths,…","""'Venice: 1202-1295', in Calend…",11,"[""1202"", ""1295""]",1265,6,"""day""",2
"""1272. Feb. 15. “Comune I.” “De…","""4. “Cambium.”Concerning mercha…","""'Venice: 1202-1295', in Calend…",2,"[""1202"", ""1295""]",1272,15,"""day""",3
"""1273. Dec. 13. “ Comune I.” “D…","""5. “Cambium.”Concerning mercha…","""'Venice: 1202-1295', in Calend…",12,"[""1202"", ""1295""]",1273,13,"""day""",4


In [13]:
t.sample(n=10)

date,text,citation,month,citation_years,year,day,precision,index
str,str,str,i64,list[str],i64,i64,str,u32
"""June 27.""","""1060. News from Italy.1. Venic…","""'Elizabeth: June 1573, 15-30',…",6,"[""1573""]",1573,27,"""day""",41246
"""May.""","""1756. English Ships taken by t…","""'Elizabeth: May 1571, 16-31', …",5,"[""1571""]",1571,1,"""month""",39827
"""June 16.Senato,Secreta.Dispacc…","""110. ALVISE CONTARINI, Venetia…","""'Venice: June 1625, 16 - 30', …",6,"[""1625""]",1625,16,"""day""",15992
"""Aug. 22. Consiglio di X. Crimi…","""407. That Ottavio Robbazzi of …","""'Venice: August 1616', in Cale…",8,"[""1616""]",1616,22,"""day""",11857
"""[July 16.]""","""324. The Queen to Throckmorton…","""'Elizabeth: July 1562, 11-20',…",7,"[""1562""]",1562,16,"""day""",31159
"""Sept. 14. MS. St. Mark's Libra…","""613. Cardinal Pole to Cardinal…","""'Venice: September 1556, 11-15…",9,"[""1556""]",1556,14,"""day""",5147
"""August.""","""441. Reformation on the Border…","""'Elizabeth: August 1561, 21-31…",8,"[""1561""]",1561,1,"""month""",30197
""" July 8. ""","""7. 'Copy of a letter written i…","""'Elizabeth: July 1579, 1-15', …",7,"[""1579""]",1579,8,"""day""",44701
"""Dec. 16.Senato,Secreta.Dispacc…","""129. CHRISTOFFORO SURIAN, Vene…","""'Venice: December 1617, 16-30'…",12,"[""1617""]",1617,16,"""day""",12486


In [14]:
t.head()

date,text,citation,month,citation_years,year,day,precision,index
str,str,str,i64,list[str],i64,i64,str,u32
"""1202. Oct.""","""1. Baldwin Count of Flanders a…","""'Venice: 1202-1295', in Calend…",10,"[""1202"", ""1295""]",1202,1,"""month""",0
"""1224. 13 Sept.""","""2. Doge Pietro Ziani and his s…","""'Venice: 1202-1295', in Calend…",9,"[""1202"", ""1295""]",1224,13,"""day""",1
"""1265. Nov. 6. “Fractus.” “Deli…","""3. Tariff of Duties on Cloths,…","""'Venice: 1202-1295', in Calend…",11,"[""1202"", ""1295""]",1265,6,"""day""",2
"""1272. Feb. 15. “Comune I.” “De…","""4. “Cambium.”Concerning mercha…","""'Venice: 1202-1295', in Calend…",2,"[""1202"", ""1295""]",1272,15,"""day""",3
"""1273. Dec. 13. “ Comune I.” “D…","""5. “Cambium.”Concerning mercha…","""'Venice: 1202-1295', in Calend…",12,"[""1202"", ""1295""]",1273,13,"""day""",4


In [15]:
t2 = t.with_columns(
    pl.when(pl.col("text").is_null())
    .then(pl.col("date"))
    .otherwise(pl.col("text"))
    .alias("text")
)

In [16]:
t2.select(pl.col("text").is_null()).sum()

text
u32
0


In [19]:
t2 = t.with_columns(
    (
        pl.when(pl.col("date").is_null()).then(pl.lit("")).otherwise(pl.col("date"))
        + pl.when(pl.col("text").is_null()).then(pl.lit("")).otherwise(pl.col("text"))
    ).alias("combined_text")
)
t2.sample(n=10)

date,text,citation,month,citation_years,year,day,precision,index,combined_text
str,str,str,i64,list[str],i64,i64,str,u32,str
""" May 17. ""","""WILKES to WALSINGHAM.After wri…","""'Elizabeth: May 1587, 11-20', …",5,"[""1587""]",1587,17,"""day""",51070,""" May 17. WILKES to WALSINGHAM.…"
"""[May 3.]""","""18. Message brought by Hans Ke…","""'Elizabeth: May 1560, 1-5', in…",5,"[""1560""]",1560,3,"""day""",28699,"""[May 3.]18. Message brought by…"
"""Dec. 21.Misc. Cod.No. 61.Venet…","""706. MARC ANTONIO PADAVIN, Ven…","""'Venice: December 1622, 16-31'…",12,"[""1622""]",1622,21,"""day""",14842,"""Dec. 21.Misc. Cod.No. 61.Venet…"
"""June 8. Sanuto Diaries, v. xxv…","""70. Giovanni Badoer and Antoni…","""'Venice: June 1520, 6-10', in …",6,"[""1520""]",1520,8,"""day""",1877,"""June 8. Sanuto Diaries, v. xxv…"
"""April 28. Senato, Secreta. Dis…","""39. Zuanne Zon, Venetian Secre…","""'Venice: April 1642', in Calen…",4,"[""1642""]",1642,28,"""day""",20725,"""April 28. Senato, Secreta. Dis…"
"""Dec. 9.""","""1470. [Throckmorton to Cecil.]…","""'Elizabeth: December 1563, 1-1…",12,"[""1563""]",1563,9,"""day""",33706,"""Dec. 9.1470. [Throckmorton to …"
"""Nov. 13. Senato, Secreta. Deli…","""338. To the Ambassador in Engl…","""'Venice: November 1637', in Ca…",11,"[""1637""]",1637,13,"""day""",20014,"""Nov. 13. Senato, Secreta. Deli…"
"""Nov. 18.""","""509. Advices.News from Rome an…","""'Elizabeth: November 1569', in…",11,"[""1569""]",1569,18,"""day""",38660,"""Nov. 18.509. Advices.News from…"
"""Nov. 21.R. O. Sadler, 1. 591. …","""305. Sadler and Croftes to Cec…","""'Elizabeth: November 1559, 21-…",11,"[""1559""]",1559,21,"""day""",27853,"""Nov. 21.R. O. Sadler, 1. 591. …"


In [20]:
import tiktoken

embedding_encoding = "cl100k_base"
encoding = tiktoken.get_encoding(embedding_encoding)
text = t2.select(pl.col("combined_text")).to_series().to_list()

text_batch = encoding.encode_batch(text)

len(text_batch)
new_col = pl.Series([len(toks) for toks in text_batch]).alias("token_length")
t2 = t2.with_columns(new_col)
t2.select(pl.col("token_length")).describe()

statistic,token_length
str,f64
"""count""",53939.0
"""null_count""",0.0
"""mean""",393.436085
"""std""",492.520189
"""min""",1.0
"""25%""",119.0
"""50%""",222.0
"""75%""",476.0
"""max""",22896.0


In [21]:
t2 = t2.with_columns(pl.col("index").cast(pl.Float64).alias("ID"))
t2.head()

date,text,citation,month,citation_years,year,day,precision,index,combined_text,token_length,ID
str,str,str,i64,list[str],i64,i64,str,u32,str,i64,f64
"""1202. Oct.""","""1. Baldwin Count of Flanders a…","""'Venice: 1202-1295', in Calend…",10,"[""1202"", ""1295""]",1202,1,"""month""",0,"""1202. Oct.1. Baldwin Count of …",230,0.0
"""1224. 13 Sept.""","""2. Doge Pietro Ziani and his s…","""'Venice: 1202-1295', in Calend…",9,"[""1202"", ""1295""]",1224,13,"""day""",1,"""1224. 13 Sept.2. Doge Pietro Z…",315,1.0
"""1265. Nov. 6. “Fractus.” “Deli…","""3. Tariff of Duties on Cloths,…","""'Venice: 1202-1295', in Calend…",11,"[""1202"", ""1295""]",1265,6,"""day""",2,"""1265. Nov. 6. “Fractus.” “Deli…",125,2.0
"""1272. Feb. 15. “Comune I.” “De…","""4. “Cambium.”Concerning mercha…","""'Venice: 1202-1295', in Calend…",2,"[""1202"", ""1295""]",1272,15,"""day""",3,"""1272. Feb. 15. “Comune I.” “De…",179,3.0
"""1273. Dec. 13. “ Comune I.” “D…","""5. “Cambium.”Concerning mercha…","""'Venice: 1202-1295', in Calend…",12,"[""1202"", ""1295""]",1273,13,"""day""",4,"""1273. Dec. 13. “ Comune I.” “D…",307,4.0


In [22]:
over_max = t2.filter(pl.col("token_length") > max_tokens).shape
total = t2.shape
print(f"Rows over {max_tokens} tokens: {over_max[0]} out of {total[0]}")

Rows over 8000 tokens: 10 out of 53939


In [27]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


def word_len(s: str):
    return len(s.split())


def explode_chunk_text(row):
    text = row["combined_text"]
    n_tokens = row["token_length"]
    id = row["ID"]
    max_tokens = 4000
    if n_tokens > max_tokens:
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=max_tokens,
            chunk_overlap=200,
            length_function=word_len,
            separators=["\n\n", "\n", " ", ""],
        )
        chunks = text_splitter.split_text(text)
        new_rows = []
        for i, chunk in enumerate(chunks):
            new_row = row.copy()
            new_row["combined_text"] = chunk
            new_row["ID"] += i / 10
            new_rows.append(new_row)
        return new_rows
    return [row]


example = (
    t2.with_columns(
        pl.struct(pl.all())
        .map_elements(
            explode_chunk_text,
            returns_scalar=True,
            return_dtype=pl.List(pl.Struct(t2.schema)),
        )
        .alias("new_rows")
    )
    .select(pl.col("new_rows"))
    .explode("new_rows")
    .unnest("new_rows")
).with_columns(
    pl.col("combined_text")
    .map_elements(lambda s: len(encoding.encode(s)))
    .alias("token_length")
)
example.head()

date,text,citation,month,citation_years,year,day,precision,index,combined_text,token_length,ID
str,str,str,i64,list[str],i64,i64,str,u32,str,i64,f64
"""1202. Oct.""","""1. Baldwin Count of Flanders a…","""'Venice: 1202-1295', in Calend…",10,"[""1202"", ""1295""]",1202,1,"""month""",0,"""1202. Oct.1. Baldwin Count of …",230,0.0
"""1224. 13 Sept.""","""2. Doge Pietro Ziani and his s…","""'Venice: 1202-1295', in Calend…",9,"[""1202"", ""1295""]",1224,13,"""day""",1,"""1224. 13 Sept.2. Doge Pietro Z…",315,1.0
"""1265. Nov. 6. “Fractus.” “Deli…","""3. Tariff of Duties on Cloths,…","""'Venice: 1202-1295', in Calend…",11,"[""1202"", ""1295""]",1265,6,"""day""",2,"""1265. Nov. 6. “Fractus.” “Deli…",125,2.0
"""1272. Feb. 15. “Comune I.” “De…","""4. “Cambium.”Concerning mercha…","""'Venice: 1202-1295', in Calend…",2,"[""1202"", ""1295""]",1272,15,"""day""",3,"""1272. Feb. 15. “Comune I.” “De…",179,3.0
"""1273. Dec. 13. “ Comune I.” “D…","""5. “Cambium.”Concerning mercha…","""'Venice: 1202-1295', in Calend…",12,"[""1202"", ""1295""]",1273,13,"""day""",4,"""1273. Dec. 13. “ Comune I.” “D…",307,4.0


In [28]:
max_tokens = 8000
example.filter(pl.col("token_length") > max_tokens)

date,text,citation,month,citation_years,year,day,precision,index,combined_text,token_length,ID
str,str,str,i64,list[str],i64,i64,str,u32,str,i64,f64


In [29]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [30]:
example.select(pl.col("token_length")).describe()

statistic,token_length
str,f64
"""count""",53980.0
"""null_count""",0.0
"""mean""",393.302408
"""std""",470.518059
"""min""",1.0
"""25%""",119.0
"""50%""",222.0
"""75%""",477.0
"""max""",7143.0


In [67]:
from scripts.cluster import embed_items

# Embed text to vectors with OpenAI
table["embeds"] = embed_items(table["text"])
table["embeds"]

0      [-0.050486620515584946, 0.01540993433445692, 0...
1      [0.02098303660750389, 0.03918666020035744, 0.0...
2      [0.014525304548442364, 0.025349894538521767, 0...
3      [0.01985478401184082, 0.003296412993222475, 0....
4      [0.007098940201103687, 0.01995459944009781, 0....
                             ...                        
234    [0.021585967391729355, 0.03303307294845581, 0....
235    [0.01747804880142212, 0.01886497065424919, 0.0...
236    [-0.008196048438549042, 0.06117403134703636, 0...
237    [0.005722603760659695, 0.020581543445587158, 0...
238    [0.005254759918898344, 0.032985880970954895, 0...
Name: embeds, Length: 239, dtype: object

In [ ]:
import numpy as np
import importlib
import scripts.cluster as cluster


# Reload the module to get latest changes
importlib.reload(cluster)
from scripts.cluster import create_cluster, spectral_clusters

In [ ]:
%%time

embeds = np.stack(table["embeds"].to_list()).astype(np.float64)
# returns cluster indexes for each item
clusters, samples, iters = create_cluster(embeds, num_clusters=25, soft_assign=True, soft_margin=0.1)
# sims, clusters

# spectral_labels = spectral_clusters(embeds, n_clusters=3, random_state=42)
# [len(c) for c in clusters]
clusters,iters

90.0
90.0
90.0
90.0
90.0
90.0
CPU times: user 664 ms, sys: 5.89 ms, total: 670 ms
Wall time: 71.7 ms


([23,
  13,
  2,
  23,
  21,
  24,
  17,
  24,
  12,
  22,
  22,
  4,
  22,
  23,
  13,
  8,
  21,
  3,
  21,
  2,
  0,
  0,
  3,
  3,
  21,
  23,
  10,
  10,
  10,
  15,
  23,
  12,
  23,
  22,
  21,
  21,
  22,
  22,
  22,
  21,
  24,
  22,
  23,
  10,
  22,
  20,
  22,
  23,
  21,
  10,
  10,
  10,
  10,
  21,
  22,
  3,
  21,
  21,
  23,
  22,
  23,
  23,
  10,
  10,
  23,
  22,
  21,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  1,
  14,
  9,
  9,
  1,
  9,
  9,
  9,
  14,
  14,
  9,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  1,
  9,
  14,
  14,
  14,
  1,
  14,
  14,
  14,
  16,
  16,
  16,
  16,
  23,
  12,
  22,
  12,
  24,
  21,
  21,
  22,
  24,
  22,
  22,
  12,
  24,
  24,
  12,
  24,
  22,
  22,
  24,
  12,
  19,
  12,
  19,
  10,
  22,
  22,
  7,
  12,
  7,
  16,
  10,
  22,
  7,
  24,
  23,
  10,
  21,
  22,
  7,
  3,
  3,
  24,
  21,
  21,
  22,
  24,
  23,
  12,
  7,
  22,
  6,
  22,
  20,
  22,
  6,
  24,
  12,
  20,
  12,
  21,
  24,
  12,
  7,
  12,
  24,
  23,
  12,

In [102]:
table["cluster"] = clusters

table["cluster"]

0      23
1      13
2       2
3      23
4      21
       ..
234    24
235    21
236    12
237     7
238    11
Name: cluster, Length: 239, dtype: int64

In [103]:
table.head()

,date,text,citation,year,embeds,cluster,month,"(month, is_month_precise)",is_month_precise,day,is_day_precise,precision
0,1202. Oct.,1. Baldwin Count of Flanders and Hainault.Nota...,"'Venice: 1202-1295', in Calendar of State Pape...",1202.0,"[-0.050486620515584946, 0.01540993433445692, 0...",23,10,"[10, True]",True,1,False,month
1,1224. 13 Sept.,2. Doge Pietro Ziani and his six Privy Counsel...,"'Venice: 1202-1295', in Calendar of State Pape...",1224.0,"[0.02098303660750389, 0.03918666020035744, 0.0...",13,9,"[9, True]",True,13,True,day
2,"1265. Nov. 6. “Fractus.” “Deliberazioni,” Gran...","3. Tariff of Duties on Cloths, Linens, and Fus...","'Venice: 1202-1295', in Calendar of State Pape...",1265.0,"[0.014525304548442364, 0.025349894538521767, 0...",2,11,"[11, True]",True,6,True,day
3,"1272. Feb. 15. “Comune I.” “Deliberazioni,” Gr...",4. “Cambium.”Concerning merchants who go (qui ...,"'Venice: 1202-1295', in Calendar of State Pape...",1272.0,"[0.01985478401184082, 0.003296412993222475, 0....",23,2,"[2, True]",True,15,True,day
4,"1273. Dec. 13. “ Comune I.” “Deliberazioni,” G...",5. “Cambium.”Concerning merchants navigating t...,"'Venice: 1202-1295', in Calendar of State Pape...",1273.0,"[0.007098940201103687, 0.01995459944009781, 0....",21,12,"[12, True]",True,13,True,day


In [104]:
cluster_samples = [table.loc[sample, "text"] for sample in samples]
cluster_samples

[20    3. Act that the Venetian ambassador in England...
 21    4. Act that an answer be sat to the Venetian a...
 Name: text, dtype: object,
 92    82. The Same to the Same.Announces the arrival...
 97    87. The Same to the Same.Requests Ludovico Gon...
 78    68. The Same to the nobleman the Captain Jacop...
 74    64. The Same to the Same.Credentials in favour...
 Name: text, dtype: object,
 19    2. Act that the money for the agreement with E...
 2     3. Tariff of Duties on Cloths, Linens, and Fus...
 Name: text, dtype: object,
 17     13. Motion made in the Grand Council and cance...
 55     45. Motions made in the Senate concerning an E...
 22     15. Motion made in the Grand Council and repea...
 23     16. Motion made in the Grand Council and repea...
 144    134. Motion made in the Senate by the two Sage...
 145    135. Motion made in the Senate by the two Sage...
 Name: text, dtype: object,
 11    The captain to have 6 livres gross per month
 Name: text, dtype: object,
 232

In [105]:
table.groupby("cluster").size()

cluster
0      2
1      4
2      2
3      6
4      1
5      2
6      3
7     11
8      1
9      7
10    13
11     6
12    22
13     2
14    23
15     1
16     5
17     1
18     4
19     2
20     4
21    24
22    44
23    27
24    22
dtype: int64

In [107]:
table.loc[table["cluster"] == 22, "text"].to_list()

['Mission to the King of England (Edward III.); notwithstanding which, galleys may fit out for the Flanders voyage, but their captain to be appointed by the Republic.',
 'Instructions for the galleys deputed to that voyage.',
 'A medical surgeon (unum medicum cirurgise) was given to the captain of the galleys; his salary being paid by the State.',
 '23. Decree of the Senate.In order that the galleys bound for the Flanders voyage may take full cargoes, and not suffer loss on account of wool brought by land, it is decreed:—“That English and Flemish wool may not be brought overland from Flanders to Venice after the calends of next March, under penalty of 25 per cent.; though one month after the return of the said galleys, wool may be brought to Venice by land.”Proclaimed by the crier (præconem), Antonio, on the 15th December.[Latin.]',
 '26. Decree of the Senate concerning four galleys appointed for the Flanders voyage.[Latin.]',
 '27. Decree of the Senate concerning the Flanders galleys.

In [ ]:
# urls = get_urls("https://www.british-history.ac.uk/cal-state-papers/venice/vol35")
# dfs = []
# for url in data_links:
#     df = parse_historical_table(url)
#     dfs.append(df)
#     print(f"finished {url}")
# main_df = pd.concat(dfs)
# display(main_df.shape)

In [ ]:
import scripts.extract_names as extract_names
import importlib

importlib.reload(extract_names)
from scripts.extract_names import get_db_people, get_db_places, Person, tag_clusters

people: list[Person] = get_db_people()
places: list[str] = get_db_places()
len(people), len(places)

(830, 79)

In [ ]:
from random import choice
from scripts.extract_names import (
    get_entities_from_text,
    ResponseSchema,
    update_people_entities,
    update_place_entities,
    config_gemini,
)

client, config = config_gemini()

num_test = 10

samples = [choice(range(len(table))) for _ in range(num_test)]

tabletest = table.iloc[samples].copy()

auth = []
recp = []
auth_loc = []
recp_loc = []
people_ents = []
place_ents = []
is_auth_loc_in_db = []
is_recp_loc_in_db = []
is_auth_in_db = []
is_recp_in_db = []

for row in tabletest.itertuples():
    row_text = row.text
    row_date = row.date
    row_text = f"{row_date}: {row_text}"
    idx = row.Index
    people = people
    place = places
    try:
        res_json, res_list = get_entities_from_text(
            row_text, people, places, client, config
        )
        people = update_people_entities(people, res_json)
        places = update_place_entities(places, res_json)
        print(f"Processed row {idx}")
        auth.append(res_json.author)
        recp.append(res_json.recipient)
        auth_loc.append(res_json.auth_location)
        recp_loc.append(res_json.recp_location)
        people_ents.append(res_json.people_entities)
        place_ents.append(res_json.place_entities)
        is_auth_loc_in_db.append(res_json.is_authloc_in_db)
        is_recp_loc_in_db.append(res_json.is_recploc_in_db)
        is_auth_in_db.append(res_json.is_auth_in_db)
        is_recp_in_db.append(res_json.is_recp_in_db)
    except Exception as e:
        print(f"Error processing row {idx}: {e} \n {e.__traceback__.tb_lineno}")
        break

Processed row 0
Processed row 1
Processed row 2
Processed row 3
Processed row 4
Processed row 5
Processed row 6
Processed row 7
Processed row 8
Processed row 9


In [113]:
tabletest["author"] = auth
tabletest["recipient"] = recp
tabletest["auth_location"] = auth_loc
tabletest["recp_location"] = recp_loc
tabletest["people_entities"] = people_ents
tabletest["place_entities"] = place_ents
tabletest["is_authloc_in_db"] = is_auth_loc_in_db
tabletest["is_recploc_in_db"] = is_recp_loc_in_db
tabletest["is_auth_in_db"] = is_auth_in_db
tabletest["is_recp_in_db"] = is_recp_in_db
tabletest

,date,text,citation,year,embeds,cluster,month,"(month, is_month_precise)",is_month_precise,day,...,author,recipient,auth_location,recp_location,people_entities,place_entities,is_authloc_in_db,is_recploc_in_db,is_auth_in_db,is_recp_in_db
0,1202. Oct.,1. Baldwin Count of Flanders and Hainault.Nota...,"'Venice: 1202-1295', in Calendar of State Pape...",1202.0,"[-0.050486620515584946, 0.01540993433445692, 0...",23,10,"[10, True]",True,1,...,Baldwin Count of Flanders and Hainault,"Markisino Soranzo, Pietro Giuliano, Marino Gra...",France,Venice,"[Baldwin Count of Flanders and Hainault, Marki...","[Ligny, Venice, Island of St. Erasmus, Flander...",True,True,False,False
1,1224. 13 Sept.,2. Doge Pietro Ziani and his six Privy Counsel...,"'Venice: 1202-1295', in Calendar of State Pape...",1224.0,"[0.02098303660750389, 0.03918666020035744, 0.0...",13,9,"[9, True]",True,13,...,Doge Pietro Ziani and his six Privy Counsellors,Agnes of Marseilles,Venice,Marseilles,"[Raymond, Giles, Agnes of Marseilles, Pietro Z...","[Venice, Marseilles, England, Paris]",True,True,False,False
2,"1265. Nov. 6. “Fractus.” “Deliberazioni,” Gran...","3. Tariff of Duties on Cloths, Linens, and Fus...","'Venice: 1202-1295', in Calendar of State Pape...",1265.0,"[0.014525304548442364, 0.025349894538521767, 0...",2,11,"[11, True]",True,6,...,Anonymous,Anonymous,Venice,Venice,[],"[England, Milan, Monza]",True,True,True,True
3,"1272. Feb. 15. “Comune I.” “Deliberazioni,” Gr...",4. “Cambium.”Concerning merchants who go (qui ...,"'Venice: 1202-1295', in Calendar of State Pape...",1272.0,"[0.01985478401184082, 0.003296412993222475, 0....",23,2,"[2, True]",True,15,...,Grand Council,Venetian Merchants and Citizens,Venice,Venice,"[merchants, Venetian]","[France, Venice]",True,True,False,False
4,"1273. Dec. 13. “ Comune I.” “Deliberazioni,” G...",5. “Cambium.”Concerning merchants navigating t...,"'Venice: 1202-1295', in Calendar of State Pape...",1273.0,"[0.007098940201103687, 0.01995459944009781, 0....",21,12,"[12, True]",True,13,...,Grand Council,Venetian Merchants and Citizens,Venice,Venice,[],"[France, Marseilles, Venice, Ottoman Empire, F...",True,True,True,True
5,1274. Oct. 9. “Comune II.” “Deliberazioni.” Gr...,6. Sterling Silver.Overseers of money gross (m...,"'Venice: 1202-1295', in Calendar of State Pape...",1274.0,"[0.001626385492272675, 0.01912439987063408, -0...",24,10,"[10, True]",True,9,...,Grand Council,Overseers of money gross,Venice,Venice,"[merchants, Giustinian]",[Venice],True,True,True,False
6,"1287. Sept. 16. “Deliberazioni,” Grand Council.",7. Saffron.[Page 3]Act passed by the Grand Cou...,"'Venice: 1202-1295', in Calendar of State Pape...",1287.0,"[0.008267330005764961, 0.04552506282925606, 0....",17,9,"[9, True]",True,16,...,Grand Council,Consuls,Venice,Venice,[],[],True,True,False,False
7,"1295. July 17. “Deliberazioni,” Grand Council....",8. Sterling Money.Whereas the Duke and Council...,"'Venice: 1202-1295', in Calendar of State Pape...",1295.0,"[0.04497085139155388, -0.005375762935727835, 0...",24,7,"[7, True]",True,17,...,Grand Council,Duke and Councillors of Candia,Venice,Crete,"[Zaneta Pilosus, Duke and Councillors of Candia]","[Candia, Crete]",True,True,True,False
8,1317–1320. “Misti Senato.” v. v. p. 37.,"9. The Voyage to Flanders.Currants and masere,...","'Venice: 1317-1399', in Calendar of State Pape...",1317.0,"[-0.004544783383607864, 0.06057766079902649, 0...",12,1,"[1, False]",False,1,...,The Senate of Venice,Venetian Merchants and Citizens,Venice,Venice,[],[Flanders],True,True,True,True
9,Ib. 172–177.,Mission to the King of England (Edward III.); ...,"'Venice: 1317-1399', in Calendar of State Pape...",1317.0,"[0.008162741549313068, 0.05200953036546707, 0....",22,1,"[1, False]",False,1,...,Republic of Venice,"Edward III, King of England",Venice,England,"[Edward III, King of England]","[England, Flanders, Venice]",True,True,True,False


In [58]:
table.loc[123, ["text", "author", "recipient", "people_entities", "place_entities"]]

text               113. Decree of the Senate.That for the benefit...
author                                          The Senate of Venice
recipient                                            King of England
people_entities    [King of England, Lord Henry of Lancaster, Ear...
place_entities                  [London, England, Lancaster, Venice]
Name: 123, dtype: object

In [57]:
table.loc[123, "text"]

'113. Decree of the Senate.That for the benefit of the galley bound to London and of our merchants, the state do write letters of recommendation to the King of England, the Duke of Lancaster, and others.[Latin, 12 lines.]'

In [59]:
table.to_csv("./test_outcomes/nov-6.csv", index=False)

In [116]:
tags = tag_clusters(client=client, cluster_text=cluster_samples)
tags

Cluster 0 tagged successfully.
Cluster 1 tagged successfully.
Cluster 2 tagged successfully.
Cluster 3 tagged successfully.
Cluster 4 tagged successfully.
Cluster 5 tagged successfully.
Cluster 6 tagged successfully.
Cluster 7 tagged successfully.
Cluster 8 tagged successfully.
Cluster 9 tagged successfully.
Cluster 10 tagged successfully.
Cluster 11 tagged successfully.
Cluster 12 tagged successfully.
An error occurred while tagging cluster 13: 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}
Cluster 14 tagged successfully.
Cluster 15 tagged successfully.
Cluster 16 tagged successfully.
Cluster 17 tagged successfully.
Cluster 18 tagged successfully.
Cluster 19 tagged successfully.
Cluster 20 tagged successfully.
Cluster 21 tagged successfully.
Cluster 22 tagged successfully.
Cluster 23 tagged successfully.
Cluster 24 tagged successfully.


['Anglo-Venetian Diplomacy, Ambassadorial Instructions, Diplomatic Finances',
 'Military Logistics, Condottiero Correspondence, Italian City-State Relations, Resource Provisioning',
 'Duties and Tariffs, Trade Regulation, Textile Commerce',
 'Anglo-Venetian Diplomacy, Trade and Commercial Affairs, Embassy Funding and Administration',
 'Wages, Compensation, Historical Payments, Employment Contracts',
 'Venetian Administration, Consular Network, Diplomatic Correspondence, Citizen Protection Abroad',
 'Venetian Senate, Maritime Trade, Flanders-London Voyages, Trade Regulation',
 'Senate Appointments, Vicecaptain Roles, London Galley Voyages, Maritime Administration',
 'Medieval Trade, Maritime Commerce, Trade Finance, Commercial Risks',
 'Detention of Janet, Financial & Property Claims, Requests to Mantuan Lord, Offers of Military Service',
 'Anglo-Venetian Relations, Royal Safe Conducts, Maritime Trade, Trade Regulations',
 'State Gifts, International Diplomacy, England-Flanders Relation

In [119]:
for idx, tag in enumerate(tags):
    print(f"Cluster {idx}: {tag}")

Cluster 0: Anglo-Venetian Diplomacy, Ambassadorial Instructions, Diplomatic Finances
Cluster 1: Military Logistics, Condottiero Correspondence, Italian City-State Relations, Resource Provisioning
Cluster 2: Duties and Tariffs, Trade Regulation, Textile Commerce
Cluster 3: Anglo-Venetian Diplomacy, Trade and Commercial Affairs, Embassy Funding and Administration
Cluster 4: Wages, Compensation, Historical Payments, Employment Contracts
Cluster 5: Venetian Administration, Consular Network, Diplomatic Correspondence, Citizen Protection Abroad
Cluster 6: Venetian Senate, Maritime Trade, Flanders-London Voyages, Trade Regulation
Cluster 7: Senate Appointments, Vicecaptain Roles, London Galley Voyages, Maritime Administration
Cluster 8: Medieval Trade, Maritime Commerce, Trade Finance, Commercial Risks
Cluster 9: Detention of Janet, Financial & Property Claims, Requests to Mantuan Lord, Offers of Military Service
Cluster 10: Anglo-Venetian Relations, Royal Safe Conducts, Maritime Trade, Trade

In [123]:
table.loc[table["cluster"] == 16, "text"].to_list()

["91. Treaty of Reconciliation between the English and German Soldiery on one side and the Italians on the other, in the service of Venice against the Genoese.Details the affray on the preceding evening at Palestrina between 7 and 8 p.m. The names of the parties to the treaty are as follow:—Sir Walter Benedict, William Cook, Englishmen, Thomas Ellis, marshal of the English, Walter Maine, Robert More, Henricus Runz, Arnoldus de Sambach, Germans, John de Basset (de Berzete) and Cantelupe (Cantelletus) of England, for themselves and in their own names, as also in the name and on behalf of all and each of of their comrades, attendants, and accomplices, whether English or of any other nation, on the one part; and on the other, Giacopo de Pepoli, Checco de' Ordelaffi, Pandolfo de Cavalcaboi, Melchior Vitaliani, Gulielmo de Lisca, Fazio Count of Bruscolo, Bernardo of Cauriaco, marshal of the Italians, Domenico de Bentivogli, Georgio de' Alidosij, Nerio de Sesummo, for themselves and in their 